In [76]:
import os
import rasterio
import geopandas as gpd
import pandas as pd
import numpy as np
from rasterio.features import shapes
from shapely.ops import unary_union
from shapely.geometry import shape, Point, Polygon, MultiPolygon, box
from rtree import index

In [85]:
def raster_to_vector(input_folder, output_file):
    # Create an empty GeoDataFrame to store all features
    all_features = []

    # Get all TIFF image files in the input folder
    tif_files = [f for f in os.listdir(input_folder) if f.endswith('.tif')]

    for tif_file in tif_files:
        input_path = os.path.join(input_folder, tif_file)

        # Open the TIFF image file
        with rasterio.open(input_path) as src:
            # Read TIFF data
            image = src.read(1)
            
            # Create a shape list to store vector features
            shapes_list = list(shapes(image, mask=None, transform=src.transform))

            # Filter out parts with a value of 1
            shapes_list = [shape(geometry) for geometry, value in shapes_list if value == 1]

            # Create a GeoDataFrame to store features of the current TIFF file
            gdf = gpd.GeoDataFrame({'geometry': shapes_list}, crs=src.crs)  # Set the coordinate reference system

            # Calculate the area of each feature and add it to the GeoDataFrame
            gdf['area_km2'] = gdf['geometry'].area / 1e6  # Convert to square kilometers

            # Filter features with an area greater than or equal to 0.1 square kilometers
            gdf = gdf[gdf['area_km2'] >= 0.1]

            # Add the features of the current TIFF file to all features
            all_features.append(gdf)

    merged_gdf = gpd.GeoDataFrame(pd.concat(all_features, ignore_index=True), crs=src.crs)

    merged_gdf['geometry'] = merged_gdf['geometry'].apply(lambda geom: geom.buffer(45, cap_style=3))

    # Spatial intersection check and merge
    merged_gdf['group'] = None
    group_count = 0

    for index, row in merged_gdf.iterrows():
        if row['group'] is None:
            # Find features that intersect with the current feature
            intersection = merged_gdf[merged_gdf['geometry'].intersects(row['geometry'])]

            if len(intersection) > 1:
                # Update group number
                group_count += 1
                merged_gdf.loc[intersection.index, 'group'] = group_count

    merged_gdf['group'] = merged_gdf['group'].fillna(0)  # Set group number to 0 for ungrouped features
    merged_gdf = merged_gdf.dissolve(by='group')
    merged_gdf['geometry'] = merged_gdf['geometry'].apply(lambda geom: geom.buffer(-40, cap_style=3))

    single_polygons = []
    for geom in merged_gdf['geometry']:
        if geom.geom_type == 'MultiPolygon':
            # Break down compound polygons into single polygons
            single_polygons.extend(list(geom.geoms)) #For shapely version<2.0, use list(geom) instead of list(geom.geoms)!!!
        else:
            single_polygons.append(geom)
    merged_gdf = gpd.GeoDataFrame({'geometry': single_polygons}, crs=merged_gdf.crs)

    for index, row in merged_gdf.iterrows():
        if row['geometry'].interiors:
            print('fill hole when raster!!!!!')
            exterior = row['geometry'].exterior
            filled_geom = Polygon(exterior)
            merged_gdf.at[index, 'geometry'] = filled_geom
    merged_gdf = merged_gdf.unary_union

    merged_gdf = gpd.GeoDataFrame(geometry=[merged_gdf], crs=src.crs)
    merged_gdf = merged_gdf.explode()
    
    filtered_gdf = merged_gdf.copy()
    for index, row in merged_gdf.iterrows():
        if row['geometry'].interiors:
            
            
            print(str(index)+'find hole when vectorize !!!!!')
            # Get the exterior ring of the polygon
            exterior = row['geometry'].exterior

            # Create a new polygon by merging the exterior and interior rings
            filled_polygon = Polygon(exterior)
            

            # Add the filled polygon to the new GeoDataFrame
            filtered_gdf.at[index, 'geometry'] = filled_polygon
        else:
            # If the polygon is invalid or has no interior rings, add the original polygon
            filtered_gdf.at[index, 'geometry'] = row['geometry']

    filtered_gdf = filtered_gdf.unary_union
    filtered_gdf = gpd.GeoDataFrame(geometry=[filtered_gdf], crs=merged_gdf.crs)
    filtered_gdf_exploded = filtered_gdf.explode()
    
    filtered_gdf_exploded.to_file(output_file, driver="GPKG")

In [90]:
input_folder =r'I:\ACM\FinalOutPut\RasterAnalysisResult_0825_2'
# input_folder =r'I:\ACM\FinalOutPut\FinalResult_0619'
# input_folder =r'I:\ACM\FinalOutPut\FinalResult_0731'
# input_folder =r'I:\ACM\FinalOutPut\FinalResult_0825'

# input det img file path  Run Four time ！！！！！！！
# output_vector = r'I:\ACM\FinalOutPut\PredictLake_0603_postprocessed.gpkg' # output gpkg file
# output_vector = r'I:\ACM\FinalOutPut\PredictLake_0619_postprocessed.gpkg' 
# output_vector = r'I:\ACM\FinalOutPut\PredictLake_0731_postprocessed.gpkg' 
output_vector = r'I:\ACM\FinalOutPut\PredictLake_0825_postprocessed.gpkg' 

raster_to_vector(input_folder, output_vector)

fill hole when raster!!!!!
fill hole when raster!!!!!
fill hole when raster!!!!!
fill hole when raster!!!!!
fill hole when raster!!!!!
fill hole when raster!!!!!
fill hole when raster!!!!!
fill hole when raster!!!!!
fill hole when raster!!!!!
fill hole when raster!!!!!
fill hole when raster!!!!!
fill hole when raster!!!!!
fill hole when raster!!!!!
fill hole when raster!!!!!
fill hole when raster!!!!!
fill hole when raster!!!!!
fill hole when raster!!!!!
fill hole when raster!!!!!
fill hole when raster!!!!!
fill hole when raster!!!!!
fill hole when raster!!!!!
fill hole when raster!!!!!
fill hole when raster!!!!!
fill hole when raster!!!!!
fill hole when raster!!!!!
fill hole when raster!!!!!
fill hole when raster!!!!!
fill hole when raster!!!!!
fill hole when raster!!!!!
fill hole when raster!!!!!
fill hole when raster!!!!!
fill hole when raster!!!!!
fill hole when raster!!!!!
fill hole when raster!!!!!
fill hole when raster!!!!!
fill hole when raster!!!!!
fill hole when raster!!!!!
f

C:\Users\authu\AppData\Local\Temp\ipykernel_21220\1341263186.py:74: FutureWarning: Currently, index_parts defaults to True, but in the future, it will default to False to be consistent with Pandas. Use `index_parts=True` to keep the current behavior and True/False to silence the warning.
  merged_gdf = merged_gdf.explode()


(0, 2159)find hole when vectorize !!!!!


C:\Users\authu\AppData\Local\Temp\ipykernel_21220\1341263186.py:97: FutureWarning: Currently, index_parts defaults to True, but in the future, it will default to False to be consistent with Pandas. Use `index_parts=True` to keep the current behavior and True/False to silence the warning.
  filtered_gdf_exploded = filtered_gdf.explode()
